In [1]:
!pip install openeo
!pip install rasterio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.8/315.8 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 206.8/206.8 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 34.4 MB/s eta 0:00:00
  Attempting uninstall: xarray
    Found existing installation: xarray 2025.7.1
    Uninstalling xarray-2025.7.1:
      Successfully uninstalled xarray-2025.7.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 73.8 MB/s eta 0:00:00


In [2]:

import rasterio
import numpy as np
import matplotlib.pyplot as plt
from datetime import date
import openeo

In [3]:
connection = openeo.connect("https://openeo.dataspace.copernicus.eu").authenticate_oidc()

Visit https://identity.dataspace.copernicus.eu/auth/realms/CDSE/device?user_code=LBRZ-YSUR 📋 to authenticate.

✅ Authorized successfully

Authenticated using device code flow.


In [5]:
#Areas de interes
lago_atitlan = {
    "west": -91.326256,
    "east": -91.07151,
    "south": 14.5948,
    "north": 14.750979
}
lago_amatitlan = {
    "west": -90.638065,
    "east": -90.512924,
    "south": 14.412347,
    "north": 14.493799
}

In [ ]:
atitlan_cube = connection.load_collection(
    "SENTINEL2_L2A",
    spatial_extent=lago_atitlan,
    temporal_extent=["2025-04-01", "2025-08-06"],
    bands=["B02", "B03", "B04", "B05", "B07", "B08", "B8A", "B11", "B12"]
)

result_graph = atitlan_cube.save_result(format="GTIFF")
job = connection.create_job(result_graph)
job.start_and_wait()
job.download_results("../data/Bandas_Atitlan_Completas.tif")


0:00:00 Job 'j-25080723075340738d53b8c8a6fb46d3': send 'start'
0:00:13 Job 'j-25080723075340738d53b8c8a6fb46d3': created (progress 0%)
0:00:18 Job 'j-25080723075340738d53b8c8a6fb46d3': queued (progress 0%)
0:00:24 Job 'j-25080723075340738d53b8c8a6fb46d3': queued (progress 0%)
0:00:32 Job 'j-25080723075340738d53b8c8a6fb46d3': queued (progress 0%)
0:00:42 Job 'j-25080723075340738d53b8c8a6fb46d3': queued (progress 0%)
0:00:55 Job 'j-25080723075340738d53b8c8a6fb46d3': queued (progress 0%)
0:01:10 Job 'j-25080723075340738d53b8c8a6fb46d3': queued (progress 0%)
0:01:30 Job 'j-25080723075340738d53b8c8a6fb46d3': running (progress N/A)
0:01:54 Job 'j-25080723075340738d53b8c8a6fb46d3': running (progress N/A)
0:02:24 Job 'j-25080723075340738d53b8c8a6fb46d3': running (progress N/A)
0:03:01 Job 'j-25080723075340738d53b8c8a6fb46d3': running (progress N/A)
0:03:48 Job 'j-25080723075340738d53b8c8a6fb46d3': running (progress N/A)


In [ ]:
import rasterio
import numpy as np
import matplotlib.pyplot as plt

# Abrir el archivo multibanda
tif_path = "../data/Bandas_Atitlan_Completas.tif"

with rasterio.open(tif_path) as src:
    bandas = src.read()
    profile = src.profile  # Guardar metadata


In [ ]:
# Asignar bandas por índice (rasterio usa base-1)
B02 = bandas[0].astype(np.float32)  # Azul
B03 = bandas[1].astype(np.float32)  # Verde
B04 = bandas[2].astype(np.float32)  # Rojo
B05 = bandas[3].astype(np.float32)  # RedEdge 1
B07 = bandas[4].astype(np.float32)  # RedEdge 3
B08 = bandas[5].astype(np.float32)  # NIR
B8A = bandas[6].astype(np.float32)  # RedEdge 4
B11 = bandas[7].astype(np.float32)  # SWIR 1
B12 = bandas[8].astype(np.float32)  # SWIR 2


In [ ]:
# NDCI: (B05 - B04) / (B05 + B04)
ndci = np.where((B05 + B04) == 0, 0, (B05 - B04) / (B05 + B04))

# Modelo cúbico de clorofila-a
chl = 826.57 * ndci**3 - 176.43 * ndci**2 + 19 * ndci + 4.071


In [ ]:
plt.figure(figsize=(10, 6))
plt.imshow(chl, cmap='viridis')
plt.colorbar(label='Clorofila-a (estimada)')
plt.title('Cianobacteria - Lago Atitlán')
plt.axis('off')
plt.show()


In [ ]:
output_path = "../data/Atitlan_Chlorophyll_Map.tif"

with rasterio.open(
    output_path,
    'w',
    driver='GTiff',
    height=chl.shape[0],
    width=chl.shape[1],
    count=1,
    dtype=chl.dtype,
    crs=profile["crs"],
    transform=profile["transform"]
) as dst:
    dst.write(chl, 1)


Amatitlan


In [ ]:


amatitlan_cube = connection.load_collection(
    "SENTINEL2_L2A",
    spatial_extent=lago_amatitlan,
    temporal_extent=["2025-04-01", "2025-08-06"],  # ajusta fechas si quieres
    bands=["B02", "B03", "B04", "B05", "B07", "B08", "B8A", "B11", "B12"]
)

result_graph = amatitlan_cube.save_result(format="GTIFF")
job = connection.create_job(result_graph)
job.start_and_wait()
job.download_results("../data/Bandas_Amatitlan_Completas.tif")

In [ ]:
import rasterio
import numpy as np
import matplotlib.pyplot as plt

tif_path = "../data/Bandas_Amatitlan_Completas.tif"  # <-- usa tu ruta real

with rasterio.open(tif_path) as src:
    arr = src.read()          # (bands, rows, cols)
    profile = src.profile     # para guardar luego con mismo CRS/transform

# Mapeo de bandas (mismo orden en que pedimos en la descarga):
B02 = arr[0].astype(np.float32)  # Azul
B03 = arr[1].astype(np.float32)  # Verde
B04 = arr[2].astype(np.float32)  # Rojo
B05 = arr[3].astype(np.float32)  # RedEdge 1
B07 = arr[4].astype(np.float32)  # RedEdge 3
B08 = arr[5].astype(np.float32)  # NIR
B8A = arr[6].astype(np.float32)  # RedEdge 4
B11 = arr[7].astype(np.float32)  # SWIR 1
B12 = arr[8].astype(np.float32)  # SWIR 2

# --- Cyano/Chl-a (como en evalscript.js) ---
# NDCI = (B05 - B04) / (B05 + B04)
ndci = np.where((B05 + B04) == 0, 0, (B05 - B04) / (B05 + B04))

# Clorofila-a (modelo cúbico del script)
chl = 826.57 * ndci**3 - 176.43 * ndci**2 + 19 * ndci + 4.071

# Visualizar
plt.figure(figsize=(10, 6))
plt.imshow(chl, cmap='viridis')
plt.colorbar(label='Clorofila-a (estimada)')
plt.title('Cianobacteria / Clorofila-a — Lago Amatitlán')
plt.axis('off')
plt.show()


In [ ]:
out_path = "../data/Amatitlan_Chlorophyll_Map.tif"

with rasterio.open(
    out_path,
    "w",
    driver="GTiff",
    height=chl.shape[0],
    width=chl.shape[1],
    count=1,
    dtype=chl.dtype,
    crs=profile["crs"],
    transform=profile["transform"]
) as dst:
    dst.write(chl, 1)

print("Guardado:", out_path)
